In [2]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("Comparison_ModelsV6/Curriculum_LearningV2_finalV6.pt", "Curriculum Learning")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle Curriculum Learning...
Calcul du log-likelihood pour Curriculum Learning...


Evaluating Curriculum Learning: 100%|██████████| 200/200 [01:31<00:00,  2.19it/s]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1417705.508148
Log-likelihood moyen par séquence: -708.852754
Log-likelihood par événement: -6.489394
Écart-type: 351.058922


In [3]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("Comparison_ModelsV6/Curriculum_Learning_finalV6.pt", "old Curriculum Learning")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle old Curriculum Learning...
Calcul du log-likelihood pour old Curriculum Learning...


Evaluating old Curriculum Learning: 100%|██████████| 200/200 [02:25<00:00,  1.37it/s]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1433822.646585
Log-likelihood moyen par séquence: -716.911323
Log-likelihood par événement: -6.563169
Écart-type: 353.659646


In [4]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("Comparison_ModelsV6/Progressive_Random_finalV6.pt", "Progressive Random")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle Progressive Random...
Calcul du log-likelihood pour Progressive Random...


Evaluating Progressive Random: 100%|██████████| 200/200 [01:31<00:00,  2.20it/s]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1458475.576305
Log-likelihood moyen par séquence: -729.237788
Log-likelihood par événement: -6.676015
Écart-type: 356.437498


In [5]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("Comparison_ModelsV6/Random_Buckets_finalV6.pt", "Random Buckets")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle Random Buckets...
Calcul du log-likelihood pour Random Buckets...


Evaluating Random Buckets: 100%|██████████| 200/200 [01:36<00:00,  2.07it/s]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1454729.476498
Log-likelihood moyen par séquence: -727.364738
Log-likelihood par événement: -6.658867
Écart-type: 355.746568


In [3]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("RetweetNetsV3/TrainNet_ep11V2.pt", "original model")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle original model...
Calcul du log-likelihood pour original model...


Evaluating original model: 100%|██████████| 200/200 [01:25<00:00,  2.34it/s]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1461968.750108
Log-likelihood moyen par séquence: -730.984375
Log-likelihood par événement: -6.692004
Écart-type: 357.604332


In [5]:
# Cellule 1: Imports et setup
import sys
sys.path.append("../")

import numpy as np
import torch as pt
import h5py
from LSTM_construction import CTLSTM
from tqdm import tqdm
import matplotlib.pyplot as plt

# Cellule 2: Charger les données de test
print("Chargement des données de test...")
with h5py.File("RetweetTestData.h5", "r") as fl:
    EventsData = np.array(fl["EventsData"])
    timesData = np.array(fl["TimesData"])
    timeMaxData = np.array(fl["TimeMaxData"])
    SeqLengthData = np.array(fl["SeqLengthData"])

print(f"Données chargées: {EventsData.shape}")

# Cellule 3: Préparation des données (même que vos notebooks existants)
N_test = EventsData.shape[0]
N_seq_Max = EventsData.shape[1]
N_types = 3
Events_one_hot = np.zeros((N_test, N_seq_Max, N_types))

for seq in range(N_test):
    for step in range(SeqLengthData[seq]):
        ev = EventsData[seq, step]
        if ev >= 0:
            Events_one_hot[seq, step, ev] = 1.0

# Conversion en tenseurs
timeScale = 1.0
EvTens = pt.tensor(Events_one_hot).double()
EvIndTens = pt.tensor(EventsData).long()
timeTensor = pt.tensor(timesData/timeScale).double()
tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
mask = EvIndTens.ge(-1+0.001)

print("Données préparées")

def evaluate_model(model_path, model_name):
    """
    Évalue un modèle et retourne les log-likelihoods par séquence
    """
    print(f"\nChargement du modèle {model_name}...")
    net = pt.load(model_path, weights_only=False)
    net.eval()
    
    BatchSize = 10
    sequence_log_likelihoods = []
    
    print(f"Calcul du log-likelihood pour {model_name}...")
    
    with pt.no_grad():
        for batchInd in tqdm(range(0, N_test, BatchSize), desc=f"Evaluating {model_name}"):
            batch_end = min(batchInd + BatchSize, N_test)
            BatchEventsHot = EvTens[batchInd:batch_end]
            BatchEventsInd = EvIndTens[batchInd:batch_end]
            BatchTimes = timeTensor[batchInd:batch_end]
            BatchTMax = tMaxTensor[batchInd:batch_end]
            BatchMask = mask[batchInd:batch_end]
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Utiliser les fonctions modifiées qui retournent les losses individuelles
            LMC, _, _, individual_mc_losses = net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates, Nsamples=1000
            )
            
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)
            
            # Log-likelihood = -individual_log_losses - individual_mc_losses
            # (car logLoss retourne déjà le négatif et MC_Loss l'intégrale positive)
            for i in range(len(individual_log_losses)):
                ll = -individual_log_losses[i] - individual_mc_losses[i]

                
                sequence_log_likelihoods.append(ll)
    
    return np.array(sequence_log_likelihoods)

# Cellule 4: Évaluation des deux modèles
curriculum_ll = evaluate_model("RetweetNetsV9/TrainNet_ep19V2.pt", "original model")
total_events = np.sum(SeqLengthData)
total_curriculum_ll = np.sum(curriculum_ll)
print("\n" + "="*60)
print("RÉSULTATS LOG-LIKELIHOOD (Équation 8)")
print("="*60)
print(f"Nombre de séquences de test: {N_test}")
print(f"Nombre total d'événements: {total_events}")
print(f"Log-likelihood total: {total_curriculum_ll:.6f}")
print(f"Log-likelihood moyen par séquence: {np.mean(curriculum_ll):.6f}")
print(f"Log-likelihood par événement: {total_curriculum_ll / total_events:.6f}")
print(f"Écart-type: {np.std(curriculum_ll):.6f}")

Chargement des données de test...
Données chargées: (2000, 264)
Données préparées

Chargement du modèle original model...
Calcul du log-likelihood pour original model...


Evaluating original model: 100%|██████████| 200/200 [04:06<00:00,  1.23s/it]


RÉSULTATS LOG-LIKELIHOOD (Équation 8)
Nombre de séquences de test: 2000
Nombre total d'événements: 218465
Log-likelihood total: -1226442.881224
Log-likelihood moyen par séquence: -613.221441
Log-likelihood par événement: -5.613910
Écart-type: 307.986081
